# Supervised Model Cross Validation

In [10]:
import pymongo
import numpy as np
import json
import import_ipynb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score

%run Introduction.ipynb
%run Background.ipynb
%run Footnotes.ipynb
%run Conclusion.ipynb

%run Classification.ipynb

In [11]:
#Connection to mongoDB
client = pymongo.MongoClient("127.0.0.1:27017")

db = client['IRsegmentationDB']
pDataset = db['pDataset']
bDataset = db['bDataset']

In [12]:
docs = [x['text'] for x in pDataset.find()][:170]
annot = [x['annotations'] for x in pDataset.find()][:170]
indexes = [x['doc'] for x in pDataset.find()][:170]

train_size = 0.8
random_state = 42

### Training Functions

In [13]:
def functional_segmenter(d, a):
    # Instatiating Introduction Classifier
    cIntro = IntroductionClassifier(d, a)
    cIntro.train_test_split(random_state, train_size)
    
    cIntro.train()
    cIntro.test()
    
    # Instantiating Background Classifier
    cBackground = BackgroundClassifier(d, a)
    cBackground.train_test_split(random_state, train_size)
    cBackground.train()
    cBackground.test()
    
    # Instatiating Footnotes Classifier
    cFootnotes = FootnotesClassifier(d, a)
    cFootnotes.train_test_split(random_state, train_size)
    cFootnotes.train()
    cFootnotes.test()
    
def conclusion_recognizer(d, a):
    cConclusion = ConclusionRecognizer(d, a)
    cConclusion.train_test_split(random_state, train_size)
    cConclusion.train()
    cConclusion.test()
    
def testing(d, a, i):
    classifier = Classification(d, a, i)
    classifier.set()
    return classifier.get_results()

### Cross Validation

In [14]:
K = 10
dim = (round(len(docs) / K))

# Dividing in Folds
data = []
tmp = []
for i in range(0, len(docs)):
    tmp.append(i)
    if(len(tmp) == dim or i == len(docs) - 1):
        data.append(tmp)
        tmp = []

In [15]:
res = []

for i in range(0, len(data)):
    train_t = [x for j, x in enumerate(data) if j != i]
    train = []
    
    for x in train_t:
        train.extend(x)
        
    docs_cv = [docs[x] for x in train]
    annot_cv = [annot[x] for x in train]
    indexes_cv = [indexes[x] for x in train]
    
    print("\n", str(i), " FOLD EVALUATION")
    
    functional_segmenter(docs_cv, annot_cv)
    conclusion_recognizer(docs_cv, annot_cv)
    
    test = data[i]
    docs_test_cv = [docs[x] for x in test]
    annot_test_cv = [annot[x] for x in test]
    indexes_test_cv = [indexes[x] for x in test]
    
    print("TESTING")
    res.append(testing(docs_test_cv, annot_test_cv, indexes_test_cv))


 0  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 1  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 2  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 3  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 4  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 5  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 6  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 7  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 8  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING

 9  FOLD EVALUATION
Start INTRODUCTION
Start BACKGROUND
Start FOOTNOTES
Start CONCLUSIONS
TESTING


#### Accuracy

In [19]:
sum([x['accuracy'] for x in res]) / len(res)

K = 10

#### CV Results

In [20]:
stats = {
    "Introduction": {"P": 0, "R": 0, "F1": 0},
    "Background": {"P": 0, "R": 0, "F1": 0},
    "Analysis": {"P": 0, "R": 0, "F1": 0},
    "Conclusion": {"P": 0, "R": 0, "F1": 0},
    "Footnotes": {"P": 0, "R": 0, "F1": 0}
}

for r in res:
    stats['Introduction']['P'] += r['Introduction']['precision']
    stats['Background']['P'] += r['Background']['precision']
    stats['Analysis']['P'] += r['Analysis']['precision']
    stats['Conclusion']['P'] += r['Conclusions']['precision']
    stats['Footnotes']['P'] += r['Footnotes']['precision']
    
    stats['Introduction']['R'] += r['Introduction']['recall']
    stats['Background']['R'] += r['Background']['recall']
    stats['Analysis']['R'] += r['Analysis']['recall']
    stats['Conclusion']['R'] += r['Conclusions']['recall']
    stats['Footnotes']['R'] += r['Footnotes']['recall']
    
    stats['Introduction']['F1'] += r['Introduction']['f1-score']
    stats['Background']['F1'] += r['Background']['f1-score']
    stats['Analysis']['F1'] += r['Analysis']['f1-score']
    stats['Conclusion']['F1'] += r['Conclusions']['f1-score']
    stats['Footnotes']['F1'] += r['Footnotes']['f1-score']

stats['Introduction']['P'] /= K
stats['Background']['P'] /= K
stats['Analysis']['P'] /= K
stats['Conclusion']['P'] /= K
stats['Footnotes']['P'] /= K

stats['Introduction']['R'] /= K
stats['Background']['R'] /= K
stats['Analysis']['R'] /= K
stats['Conclusion']['R'] /= K
stats['Footnotes']['R'] /= K

stats['Introduction']['F1'] /= K
stats['Background']['F1'] /= K
stats['Analysis']['F1'] /= K
stats['Conclusion']['F1'] /= K
stats['Footnotes']['F1'] /= K

print(json.dumps(stats, indent=4))

{
    "Introduction": {
        "P": 1.0,
        "R": 1.0,
        "F1": 1.0
    },
    "Background": {
        "P": 0.9588578088578089,
        "R": 0.5834558823529413,
        "F1": 0.7192845542253096
    },
    "Analysis": {
        "P": 0.8511421795411568,
        "R": 0.9470352561979279,
        "F1": 0.8957136614279431
    },
    "Conclusion": {
        "P": 0.9405076560615504,
        "R": 0.9127157137401009,
        "F1": 0.9254208350706223
    },
    "Footnotes": {
        "P": 0.990909090909091,
        "R": 0.9467088467088466,
        "F1": 0.9662838827838829
    }
}


In [ ]:
with open(f'./eval/CV01.p', 'wb') as handle:
        pickle.dump(stats, handle, protocol=pickle.HIGHEST_PROTOCOL)